# Adding Dashboards via API

* This notebook walks through how we can use Fiddler's API to create/add a dashboard to the platform, in the desired chart arrangement order.
* This workflow assumes the model has already been onboarded and charts have been created (either manually through the UI or through the API, see `add_charts.ipynb`)

**Note: this API is not officially supported and lacks the same guarantees around backwards compatibility that our other APIs have.**

In [ ]:
import time as time

import fiddler as fdl

from fiddler_utils import get_or_init

# from __future__ import annotations

import logging

from requests import HTTPError, Response
from fiddler.libs.http_client import RequestClient

print(f"Running Fiddler Python client version {fdl.__version__}")

Running Fiddler Python client version 3.10.0


In [ ]:
URL = ""  # Make sure to include the full URL (including https://).
TOKEN = ""

# Enter name of project and model to retrieve
PROJECT_NAME = ""
MODEL_NAME = ""

URL = ""  # e.g., 'https://target.fiddler.ai'
TOKEN = ""
PROJECT_NAME = ""
MODEL_NAME = ""

In [6]:
# Initialize connection
get_or_init(url=URL, token=TOKEN, log_level="ERROR")

In [ ]:
project = fdl.Project.get_or_create(name=PROJECT_NAME)
model = fdl.Model.from_name(name=MODEL_NAME, project_id=project.id)

In [ ]:
# Run this for dashboard API function definitions
logger = logging.getLogger(__name__)

client = RequestClient(
    URL,
    headers={
        "Content-Type": "application/json",
        "Authorization": f"Bearer {TOKEN}",
    },
)


def create_dashboard(
    project_name: str,
    model_name: str,
    charts: list,
    dashboard: dict,
) -> dict:
    dashboards_url = "v2/dashboards"

    dashboard["organization_name"] = "test"
    dashboard["project_name"] = project_name

    try:
        # Map the chart titles to uuids
        chart_titles = {chart["title"]: chart["id"] for chart in charts}
        # Get model name from yaml config
        model_name = dashboard.get("model_name", None)
        dashboard.pop("model_name", None)

        # Replace chart titles with uuids in dashboard yaml
        for index, saved_chart in enumerate(dashboard.get("layouts")):
            chart_title = saved_chart.get("chart_title")
            saved_chart["chart_uuid"] = chart_titles.get(chart_title)
            saved_chart.pop("chart_title", None)

        dashboard_resp: Response = client.post(url=dashboards_url, data=dashboard)

        project = fdl.Project.get_or_create(name=project_name)
        model = fdl.Model.from_name(name=model_name, project_id=project.id)

        # Set the created dashboard as default
        default_dashboard_url = f"v3/models/{model.id}/default-dashboard"

        payload = {}
        payload["dashboard_uuid"] = dashboard_resp.json()["data"].get("uuid")

        client.put(url=default_dashboard_url, data=payload)

    except HTTPError as hex:
        logger.error(
            f"HTTPError occured: {hex.response.text} with error code {hex.response.status_code}"
        )
        raise hex


def create_dashboards(project_name: str, model_name: str, dashboards: list) -> None:
    # get list of charts that needed to create the dashboard
    charts_url = f"/v3/charts?filter=%7B%22condition%22:%22AND%22,%22rules%22:[%7B%22field%22:%22project_name%22,%22operator%22:%22equal%22,%22value%22:%22{project_name}%22%7D]%7D&search=&offset=0&limit=40"

    try:
        charts_response = client.get(url=charts_url)

        charts = charts_response.json().get("data").get("items")
    except Exception as ex:
        logger.exception("Issue fetching existing charts for charts and response")
        raise ex

    for dashboard in dashboards:
        try:
            create_dashboard(project_name, model_name, charts, dashboard)
        except Exception as ex:
            logger.exception(ex)


In [ ]:
# Gather all the charts for specified PROJECT_NAME

charts_url = f"/v3/charts?filter=%7B%22condition%22:%22AND%22,%22rules%22:[%7B%22field%22:%22project_name%22,%22operator%22:%22equal%22,%22value%22:%22{PROJECT_NAME}%22%7D]%7D&search=&offset=0&limit=40"
charts_response = client.get(url=charts_url)
charts = charts_response.json().get("data").get("items")

In [ ]:
# List out chart titles (out of order)

chart_titles = {chart["title"]: chart["id"] for chart in charts}
chart_titles

In [ ]:
# Dashboards definition with chart order (YAML version below)

dashboards_list = [
    {
        "layouts": [
            {
                "chart_title": "Airline Delay Model Overview",
                "grid_props": {
                    "height": 1,
                    "position_x": 0,
                    "position_y": 0,
                    "width": 1,
                },
            },
            {
                "chart_title": "Actual vs. Predicted Arrival Delay",
                "grid_props": {
                    "height": 1,
                    "position_x": 1,
                    "position_y": 0,
                    "width": 1,
                },
            },
            {
                "chart_title": "Mean Absolute Error",
                "grid_props": {
                    "height": 1,
                    "position_x": 0,
                    "position_y": 1,
                    "width": 1,
                },
            },
            {
                "chart_title": "MAE by Destination",
                "grid_props": {
                    "height": 1,
                    "position_x": 1,
                    "position_y": 1,
                    "width": 1,
                },
            },
            {
                "chart_title": "Data Feature Drift",
                "grid_props": {
                    "height": 1,
                    "position_x": 0,
                    "position_y": 2,
                    "width": 1,
                },
            },
            {
                "chart_title": "Potential Arrival Delay Costs",
                "grid_props": {
                    "height": 1,
                    "position_x": 1,
                    "position_y": 2,
                    "width": 1,
                },
            },
            {
                "chart_title": "DI Violations",
                "grid_props": {
                    "height": 1,
                    "position_x": 0,
                    "position_y": 3,
                    "width": 1,
                },
            },
            {
                "chart_title": "Pre-Production Prediction Scatterplot",
                "grid_props": {
                    "height": 1,
                    "position_x": 1,
                    "position_y": 3,
                    "width": 1,
                },
            },
            {
                "chart_title": "Production Prediction Scatterplot",
                "grid_props": {
                    "height": 1,
                    "position_x": 0,
                    "position_y": 4,
                    "width": 1,
                },
            },
            {
                "chart_title": "Correlation between Departure and Arrival Delay",
                "grid_props": {
                    "height": 1,
                    "position_x": 1,
                    "position_y": 4,
                    "width": 1,
                },
            },
        ],
        "model_name": MODEL_NAME,
        "options": {
            "filters": {"time_label": "6m", "time_zone": "America/Los_Angeles"}
        },
        "organization_name": "test",
        "project_name": PROJECT_NAME,
        "title": "test_dashboard",
    }
]


In [ ]:
# Dashboards ordering YAML config (uncomment and save as .yaml, or use export/import below)

# dashboards:
#   - layouts:
#     - chart_title: Airline Delay Model Overview
#       grid_props:
#         height: 1
#         position_x: 0
#         position_y: 0
#         width: 1
#     - chart_title: Actual vs. Predicted Arrival Delay
#       grid_props:
#         height: 1
#         position_x: 1
#         position_y: 0
#         width: 1
#     - chart_title: Mean Absolute Error
#       grid_props:
#         height: 1
#         position_x: 0
#         position_y: 1
#         width: 1
#     - chart_title: MAE by Destination
#       grid_props:
#         height: 1
#         position_x: 1
#         position_y: 1
#         width: 1
#     - chart_title: Data Feature Drift
#       grid_props:
#         height: 1
#         position_x: 0
#         position_y: 2
#         width: 1
#     - chart_title: Potential Arrival Delay Costs
#       grid_props:
#         height: 1
#         position_x: 1
#         position_y: 2
#         width: 1
#     - chart_title: DI Violations
#       grid_props:
#         height: 1
#         position_x: 0
#         position_y: 3
#         width: 1
#     - chart_title: Pre-Production Prediction Scatterplot
#       grid_props:
#         height: 1
#         position_x: 1
#         position_y: 3
#         width: 1
#     - chart_title: Production Prediction Scatterplot
#       grid_props:
#         height: 1
#         position_x: 0
#         position_y: 4
#         width: 1
#     - chart_title: Correlation between Departure and Arrival Delay
#       grid_props:
#         height: 1
#         position_x: 1
#         position_y: 4
#         width: 1
#     options:
#       filters:
#         time_label: 6m
#         time_zone: America/Los_Angeles
#     organization_name: "test"
#     project_name: PROJECT_NAME
#     model_name: arrival_delay_regression
#     title: "test_dashboard"

In [ ]:
# Optional export/import from YAML

import yaml

PATH_TO_DASHBOARD_YAML = "dashboards.yaml"

with open(PATH_TO_DASHBOARD_YAML, "w") as stream:
    try:
        data = {"dashboards": dashboards_list}
        yaml.safe_dump(data, stream)
    except yaml.YAMLError as exc:
        print(exc)

with open(PATH_TO_DASHBOARD_YAML, "r") as stream:
    try:
        dashboards_list = yaml.safe_load(stream)["dashboards"]
    except yaml.YAMLError as exc:
        print(exc)


In [ ]:
# Run command to create dashboard(s) according to order in the yaml definition

create_dashboards(
    project_name=PROJECT_NAME, model_name=MODEL_NAME, dashboards=dashboards_list
)